In [ ]:
from loica import *
import matplotlib.pyplot as plt
import numpy as np
import getpass

In [ ]:
from flapjack import *
#fj = Flapjack(url_base='flapjack.rudge-lab.org:8000')
fj = Flapjack(url_base='localhost:8000')
fj.log_in(username=input('Flapjack username: '), password=getpass.getpass('Password: '))

In [ ]:
study = fj.create('study', name='Loica testing', description='Test study for demonstrating Loica')

In [ ]:
dna = fj.create('dna', name='source')
vector = fj.create('vector', name='source', dnas=dna.id)

In [ ]:
network = GeneticNetwork(vector=vector.id[0])

In [ ]:
sfp = fj.create('signal', name='SFP', color='green', description='Simulated fluorescent protein')
reporter = Reporter(name='SFP', color='green', degradation_rate=0, init_concentration=0, signal_id=sfp.id[0])

In [ ]:
network.add_reporter(reporter)

The source operator is a simple constitutive gene, which outputs a constant expression rate $\phi = C$.

In [ ]:
source = Source(output=reporter, rate=10)
network.add_operator(source)

In [ ]:
plt.figure(figsize=(3,3), dpi=150)
network.draw()

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass(t):
    return gompertz(t, 0.01, 1, 1, 0.5)

metab = SimulatedMetabolism(biomass, growth_rate)

In [ ]:
media = fj.create('media', name='loica', description='Simulated loica media')
strain = fj.create('strain', name='loica', description='Loica test strain')

samples = []
for _ in range(1):
    sample = Sample(genetic_network=network, 
                    metabolism=metab,
                    media=media.id[0],
                    strain=strain.id[0])
    samples.append(sample)

In [ ]:
biomass_signal = fj.create('signal', name='SOD', description='Simulated OD', color='black')
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.24,
              name='Loica constitutive expression (no noise)',
              description='Simulated constitutive gene generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run(nsr=0.01) # 10% noise

In [ ]:
assay.upload(fj, study=study.id[0])

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='SFP'].plot(x='Time', y='Measurement', style='.', ax=ax)

In [ ]:
source.characterize(
    fj,
    vector=vector.id,
    media=media.id,
    strain=strain.id,
    signal=sfp.id,
    biomass_signal=biomass_signal.id
)

In [ ]:
source.rate